In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,80797,+4,3170.0,1.0,62876.0,14751,4257.0,56.1
1,Italy,15113,"+2,651",1016.0,189.0,1258.0,12839,1153.0,250.0
2,Iran,10075,"+1,075",429.0,75.0,3276.0,6370,0.0,120.0
3,S. Korea,7979,+224,67.0,7.0,510.0,7402,93.0,155.6
4,Spain,3146,+869,86.0,31.0,189.0,2871,190.0,67.3
...,...,...,...,...,...,...,...,...,...
123,St. Barth,1,0,0.0,0.0,0.0,1,0.0,0.0
124,St. Vincent Grenadines,1,0,0.0,0.0,0.0,1,0.0,0.0
125,Togo,1,0,0.0,0.0,0.0,1,0.0,0.1
126,Trinidad and Tobago,1,+1,0.0,0.0,0.0,1,0.0,0.7


In [6]:
# storing the rate of growth for analysis

#importing previous data
growth_last = pd.read_csv("growth.csv")

#staging new data to add
growth_df = pd.DataFrame()


growth_df["Date"]=[0]
growth_df["NewCases"]=[0]
growth_df["TotalCases"] =[0]
growth_df["VarianceMultiple"]=[0]
growth_df["GrowthFactor"]=[0]

growth_df["Date"]= dt.datetime.today()
growth_df["NewCases"] = corona_df["NewCases"].sum()
growth_df["TotalCases"] =corona_df["TotalCases"].sum()
growth_df["VarianceMultiple"]= 1 + (growth_df["NewCases"]/growth_df["TotalCases"])
growth_df["GrowthFactor"]=[growth_df["NewCases"][0]/growth_last["NewCases"][len(growth_last)-1]]

#appending new data 
updated_growth_df = growth_last.append(growth_df)
updated_growth_df.reset_index(inplace=True,drop=True)
# #updated_growth_df.to_csv('growth.csv',index = False)
updated_growth_df.to_html('growth.html',index = False)
updated_growth_df.to_json('growth.json',orient = "records")
updated_growth_df

TypeError: can only concatenate str (not "int") to str

In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)

#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)



df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Longitude,Latitude
0,China,80797,4,3170.0,1.0,62876.0,14751,4257.0,56.1,103.819074,36.561765
1,Italy,15113,2651,1016.0,189.0,1258.0,12839,1153.0,250.0,12.070013,42.796626
2,Iran,10075,1075,429.0,75.0,3276.0,6370,0.0,120.0,54.274070,32.575033
3,S. Korea,7979,224,67.0,7.0,510.0,7402,93.0,155.6,127.839161,36.385240
4,Spain,3146,869,86.0,31.0,189.0,2871,190.0,67.3,-3.647550,40.244487
...,...,...,...,...,...,...,...,...,...,...,...
109,Bhutan,1,0,0.0,0.0,0.0,1,0.0,0.0,90.401882,27.411066
110,Mongolia,1,0,0.0,0.0,0.0,1,0.0,0.3,103.052998,46.826815
111,Togo,1,0,0.0,0.0,0.0,1,0.0,0.1,0.962328,8.525314
112,Trinidad and Tobago,1,1,0.0,0.0,0.0,1,0.0,0.7,-61.265679,10.457334


In [8]:
df_used.to_csv('data.csv')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"China","TotalCases":80797,"NewCases":"4","TotalDeaths":3170.0,"NewDeaths":1.0,"TotalRecovered":62876.0,"ActiveCases":14751,"CriticalCases":4257.0,"Tot\\u00a0Cases\\/1M pop":56.1,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":15113,"NewCases":"2651","TotalDeaths":1016.0,"NewDeaths":189.0,"TotalRecovered":1258.0,"ActiveCases":12839,"CriticalCases":1153.0,"Tot\\u00a0Cases\\/1M pop":250.0,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"Iran","TotalCases":10075,"NewCases":"1075","TotalDeaths":429.0,"NewDeaths":75.0,"TotalRecovered":3276.0,"ActiveCases":6370,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":120.0,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"S. Korea","TotalCases":7979,"NewCases":"224","TotalDeaths":67.0,"NewDeaths":7.0,"TotalRecovered":510.0,"ActiveCases":7402,"CriticalCases":93.0,"Tot\\u00a0Cases\\/1M pop":155.6,"Longitude":127.8391609,"Latitude":36.38523983},{"Country":"Spain","TotalCases":3146,"NewCase